In [44]:
import pandas as pd #importing the Pandas Library as 'pd'
from keplergl import KeplerGl #importing KeplerGl
import geopandas as gpd #importing geopandas as 'gpd'
from shapely.geometry import Point
import xml.etree.ElementTree as ET

from geopy.geocoders import Nominatim # Imports the geopy for the geocoding

In [45]:

map = KeplerGl(height=600, width=800)

User Guide: https://docs.kepler.gl/docs/keplergl-jupyter


In [46]:
filename = "Liverpool.geojson"
file = open(filename)
df = gpd.read_file(file)

In [47]:
df

,LSOA11CD,LSOA11NM,LSOA code (2011),LSOA name (2011)_x,Local Authority District code (2019)_x,Local Authority District name (2019)_x,Total population: mid 2015 (excluding prisoners),Dependent Children aged 0-15: mid 2015 (excluding prisoners),Population aged 16-59: mid 2015 (excluding prisoners),Older population aged 60 and over: mid 2015 (excluding prisoners),...,Local Authority District name (2019),Housing in poor condition indicator,Houses without central heating indicator,Road traffic accidents indicator,Nitrogen dioxide (component of air quality indicator),Benzene (component of air quality indicator),Sulphur dioxide (component of air quality indicator),Particulates (component of air quality indicator),Air quality indicator,geometry
0,Liverpool 050G,Liverpool 050G,E01032506,Liverpool 050G,Liverpool 050G,E08000012,326,208,0,0,...,Liverpool 050G,0.334,0.061,0.0,0.0,0.12,0.49,0.0,1.04,"POLYGON ((-2.95426 53.38099, -2.95408 53.38093..."
1,Liverpool 050H,Liverpool 050H,E01032507,Liverpool 050H,Liverpool 050H,E08000012,233,183,0,0,...,Liverpool 050H,0.392,0.053,0.0,0.0,0.12,0.49,0.0,1.04,"POLYGON ((-2.95186 53.38009, -2.95182 53.38007..."
2,Liverpool 050F,Liverpool 050F,E01032505,Liverpool 050F,Liverpool 050F,E08000012,295,233,0,0,...,Liverpool 050F,0.138,0.041,0.0,0.0,0.12,0.49,0.0,1.04,"POLYGON ((-2.96348 53.37651, -2.96314 53.37672..."
3,Liverpool 027G,Liverpool 027G,E01032508,Liverpool 027G,Liverpool 027G,E08000012,335,344,0,0,...,Liverpool 027G,0.277,0.067,0.0,0.0,0.20,0.59,0.0,1.31,"POLYGON ((-2.91993 53.41418, -2.91991 53.41425..."
4,Liverpool 027H,Liverpool 027H,E01032509,Liverpool 027H,Liverpool 027H,E08000012,238,255,0,0,...,Liverpool 027H,0.252,0.103,0.0,0.0,0.15,0.57,0.0,1.18,"POLYGON ((-2.92471 53.41143, -2.92470 53.41143..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
293,Liverpool 012A,Liverpool 012A,E01006611,Liverpool 012A,Liverpool 012A,E08000012,214,230,0,0,...,Liverpool 012A,0.344,0.119,0.0,0.0,0.22,0.61,0.0,1.40,"POLYGON ((-2.97110 53.43924, -2.97103 53.43949..."
294,Liverpool 014A,Liverpool 014A,E01006699,Liverpool 014A,Liverpool 014A,E08000012,315,322,0,0,...,Liverpool 014A,0.210,0.042,0.0,0.0,0.22,0.61,0.0,1.40,"POLYGON ((-2.97750 53.43998, -2.97755 53.43996..."
295,Liverpool 016C,Liverpool 016C,E01006618,Liverpool 016C,Liverpool 016C,E08000012,441,233,0,0,...,Liverpool 016C,0.206,0.066,0.0,0.0,0.18,0.55,0.0,1.22,"POLYGON ((-2.92479 53.43070, -2.92477 53.43072..."
296,Liverpool 012D,Liverpool 012D,E01006698,Liverpool 012D,Liverpool 012D,E08000012,290,398,0,0,...,Liverpool 012D,0.276,0.079,0.0,0.0,0.22,0.61,0.0,1.40,"POLYGON ((-2.97537 53.44391, -2.97537 53.44391..."


In [48]:
map.add_data(data=df, name="Liverpool_lsoa")



In [49]:
! pip3 install geopy

In [50]:
# Notes on terms
# Select a type polygon
# Select geo as geometry
# Strokes refer to the width of the line
# Fill color changes the color of the polygons
# Edit the 'new layer' to change the names of the
# Press the eye icon in the exported widget to bring the layer to the top


In [51]:
# Initialises the geocoding
geolocator = Nominatim(user_agent="Liverpool_analysis")


In [52]:
# The geocoding function

def long_lat_func(x):
  try:
    location = geolocator.geocode(str(x))
    # Parse the tuple
    return location.latitude , location.longitude
  except:
    return None, None

# Adding in the restaurant locations

In [54]:
# Example of the xml that is being parsed
'''
<Header>
    <ExtractDate>2021-01-27</ExtractDate>
    <ItemCount>3959</ItemCount>
    <ReturnCode>Success</ReturnCode>
  </Header>
  <EstablishmentCollection>
    <EstablishmentDetail>
      <FHRSID>1264123</FHRSID>
      <LocalAuthorityBusinessID>78306</LocalAuthorityBusinessID>
      <BusinessName>@Desserts</BusinessName>
      <BusinessType>Takeaway/sandwich shop</BusinessType>
      <BusinessTypeID>7844</BusinessTypeID>
      <AddressLine1>17, Central Shopping Centre Ranelagh Street</AddressLine1>
      <AddressLine3>Liverpool</AddressLine3>
      <AddressLine4>Liverpool</AddressLine4>
      <PostCode>L1 1QE</PostCode>
      <RatingValue>AwaitingInspection</RatingValue>
      <RatingKey>fhrs_awaitinginspection_en-GB</RatingKey>
      <RatingDate xsi:nil="true"/>
      <LocalAuthorityCode>414</LocalAuthorityCode>
      <LocalAuthorityName>Liverpool</LocalAuthorityName>
      <LocalAuthorityWebSite>http://www.liverpool.gov.uk</LocalAuthorityWebSite>
      <LocalAuthorityEmailAddress>environmental.health@liverpool.gov.uk</LocalAuthorityEmailAddress>
      <Scores/>
      <SchemeType>FHRS</SchemeType>
      <NewRatingPending>False</NewRatingPending>
      <Geocode>
      
      '''

IndentationError: unindent does not match any outer indentation level (<tokenize>, line 6)

In [55]:
tree = ET.parse('Liverpool.xml')
root = tree.getroot()



df_cols = ["LocalAuthorityBusinessID", "BusinessName","BusinessAddress","BusinessType","BusinessTypeID", "RatingKey", "Postcode"]
rows = []

food_outlets_df  = pd.DataFrame(rows, columns = df_cols)

for neighbor in root.iter('EstablishmentCollection'):
   for node in neighbor:
       bus_laid=node.find("LocalAuthorityBusinessID").text  if node is not None else None
       bus_rate=node.find("RatingKey").text if node is not None else None  
       bus_address=""
       try:
        bus_address=bus_address+" "+node.find('AddressLine1').text 
       except:
        bus_address=bus_address     
       try:
        bus_address=bus_address+" "+node.find('AddressLine2').text 
       except:
        bus_address=bus_address 
       try:
        bus_address=bus_address+" "+node.find('AddressLine3').text 
       except:
        bus_address=bus_address
       try:
         bus_postcode=node.find("PostCode").text if node is not None else None
         bus_address=bus_address+" "+bus_postcode+ " "+"UK" 
       except:
          bus_postcode=("missing")    
       bus_type=node.find("BusinessType").text if node is not None else None
       bus_name=node.find("BusinessName").text  if node is not None else None
       bus_type_id=node.find("BusinessTypeID").text if node is not None else None
       try:
         bus_postcode=node.find("PostCode").text if node is not None else None
       except:
          bus_postcode=("missing") 
       rows.append({"LocalAuthorityBusinessID": bus_laid, "BusinessName": bus_name, "BusinessAddress": bus_address,
                  "BusinessType": bus_type,"BusinessTypeID":bus_type_id, "Postcode": bus_postcode, "RatingValue": bus_rate})

food_outlets_df = pd.DataFrame(rows, columns = df_cols)

food_outlets_df.head(n=5)

,LocalAuthorityBusinessID,BusinessName,BusinessAddress,BusinessType,BusinessTypeID,RatingKey,Postcode
0,78306,@Desserts,"17, Central Shopping Centre Ranelagh Street L...",Takeaway/sandwich shop,7844,NaN,L1 1QE
1,71900,08 Burger Unit,,Mobile caterer,7846,NaN,missing
2,79155,109 Allerton,109 Allerton Road Mossley Hill Liverpool L18 ...,Restaurant/Cafe/Canteen,1,NaN,L18 2DD
3,78939,14forty,100 Wavertree Boulevard Liverpool L7 9PT UK,Restaurant/Cafe/Canteen,1,NaN,L7 9PT
4,PI/000272350,2 in 1,54 Durning Road Liverpool L7 5NG UK,Takeaway/sandwich shop,7844,NaN,L7 5NG


In [56]:
food_outlets_df["BusinessType"].value_counts()

Restaurant/Cafe/Canteen                  1152
Retailers - other                         741
Takeaway/sandwich shop                    685
Pub/bar/nightclub                         388
Other catering premises                   229
Hospitals/Childcare/Caring Premises       198
School/college/university                 186
Retailers - supermarkets/hypermarkets     124
Mobile caterer                            117
Hotel/bed & breakfast/guest house          87
Distributors/Transporters                  34
Manufacturers/packers                      18
Name: BusinessType, dtype: int64

In [57]:
# Applies the geocoding function

food_outlets_df["lat"] , food_outlets_df["lon"] = zip(*food_outlets_df["BusinessAddress"].apply(long_lat_func))

In [58]:
food_outlets_df.head(n=5)

,LocalAuthorityBusinessID,BusinessName,BusinessAddress,BusinessType,BusinessTypeID,RatingKey,Postcode,lat,lon
0,78306,@Desserts,"17, Central Shopping Centre Ranelagh Street L...",Takeaway/sandwich shop,7844,NaN,L1 1QE,None,None
1,71900,08 Burger Unit,,Mobile caterer,7846,NaN,missing,None,None
2,79155,109 Allerton,109 Allerton Road Mossley Hill Liverpool L18 ...,Restaurant/Cafe/Canteen,1,NaN,L18 2DD,53.3818,-2.90548
3,78939,14forty,100 Wavertree Boulevard Liverpool L7 9PT UK,Restaurant/Cafe/Canteen,1,NaN,L7 9PT,53.4057,-2.93406
4,PI/000272350,2 in 1,54 Durning Road Liverpool L7 5NG UK,Takeaway/sandwich shop,7844,NaN,L7 5NG,53.4064,-2.94843


In [59]:
# Subsets to drop the na terms
food_outlets_df=food_outlets_df.dropna(subset=['lon', 'lat'])


# Subsets to drop the na terms
food_outlets_df =food_outlets_df[food_outlets_df["Postcode"]!="Missing"]

#Converts the longitude and latitude as points data to give the geocoordinates for the data frame
geometry = [Point(xy) for xy in zip(food_outlets_df['lon'], food_outlets_df['lat'])]



# Create a GeoDataFrame from art and verify the type
food_outlets_geo = gpd.GeoDataFrame(food_outlets_df, crs = 'epsg:4326' , geometry = geometry)

map.add_data(data=food_outlets_geo, name="food_outlets")

# Adding in the foodbank information

In [60]:
foodbanks=pd.read_csv("Liverpool_foodbanks_geocode_input.csv")

In [61]:
foodbanks.head(n=3)

,Category,Company Name,Region,name,Street Address,City,State,Zip,Mobile Number,Website Address,Email,Reviews Average,Count Of Review
0,Charity,The Trussell Trust Food Bank,Liverpool,60 Peel Rd Bootle L20 4RW,60 Peel RdBootle L20 4RW,60 Peel Rd,BootleL20,4.0,0151 933 1300,trusselltrust.org,reg_with_fund_reg@2x.png,5.0,5.0
1,Foodbank,Christ Church Community Centre,Liverpool,8 Oxford Rd Bootle L20 9HW,8 Oxford RdBootle L20 9HW,8 Oxford Rd,BootleL20,9.0,NaN,NaN,NaN,NaN,NaN
2,Church,St Oswalds Church Centre,Liverpool,Ronald Ross Ave Netherton Bootle L30 5RD,Ronald Ross AveNetherton,Liverpool,BootleL30,5.0,0151 521 6002,NaN,NaN,3.0,3.0


In [62]:
# Adjust the coding of the address to make the geocoding more accurate

def complete_address(x):
  if "Bootle" in x:
    return x+", Bootle, UK"
  if "Waterloo" in x:
    return x+", Waterloo, UK"
  if "Speke" in x:
    return x+", Speke, UK"
  else:
    return x+", Liverpool, UK"


foodbanks["full_address"]=foodbanks["name"].apply(complete_address)

In [63]:
# Applies the geocoding function

foodbanks["lat"] , foodbanks["lon"] = zip(*foodbanks["full_address"].apply(long_lat_func))

In [64]:
# Subsets to drop the na terms
foodbanks=foodbanks.dropna(subset=['lon', 'lat'])

# Foodbanks drop Bootle
foodbanks=foodbanks[~foodbanks["full_address"].str.contains('Bootle')]

#Converts the longitude and latitude as points data to give the geocoordinates for the data frame
geometry = [Point(xy) for xy in zip(foodbanks['lon'], foodbanks['lat'])]



# Create a GeoDataFrame from art and verify the type
foodbanks_geo = gpd.GeoDataFrame(foodbanks, crs = 'epsg:4326' , geometry = geometry)


# Foodbanks drop Bootle



In [65]:
# Selects the points that fall within Liverpool

PointInPoly = gpd.sjoin(foodbanks_geo,df, how='left',op='within') 

In [66]:
# Create a GeoDataFrame from art and verify the type
foodbanks_geo = gpd.GeoDataFrame(PointInPoly, crs = 'epsg:4326' , geometry = geometry)


In [67]:
map.add_data(data=foodbanks_geo, name="foodbanks")

# The map

In [68]:
map

KeplerGl(data={'Liverpool_lsoa': {'index': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, …

In [41]:
# Saves the map to an html format that can be rendered in the browser
map.save_to_html(file_name='Liverpool_15_12_20.html')

Map saved to Liverpool_15_12_20.html!


In [ ]:
The map can also be served as a flask app

from flask import Flask

app = Flask(__name__)

@app.route('/')
def index():
    return map_1._repr_html_()

if __name__ == '__main__':
    app.run(debug=True)

In [42]:
# The code below is a way to speed up the geocoding

In [ ]:
import multiprocessing
from geopy.geocoders import Nominatim
geocoder = Nominatim()
def worker(address):
    return geocoder.geocode(address)
pool = multiprocessing.Pool(processes=multiprocessing.cpu_count())
addresses = ['58 Victoria Embankment, London', 'Ferry Road, Edinburgh']
locations = pool.map(worker, addresses)